In [17]:
# Indexing script for Compact Memory fulltext search
import os, json, pprint, elasticsearch, time, sys, requests
base = '/data/cm/output/'

In [18]:
c = 0
chunk = 0
data = []

for journal in os.listdir(base+'es_meta_fra/'):
    
    print('Processing', journal, 'Chunk', str(chunk), end=' ')
    
    
    for file in os.listdir(base+'es_meta_fra/'+journal+'/'):
        
        f = open(base+'es_meta_fra/'+journal+'/'+file, 'r', encoding='utf-8').readlines()
        
            
        header = json.loads(f[0].strip('\n'))
        body = json.loads(f[1].strip('\n'))
        
        
            
        data.append(json.dumps({"index": header}, ensure_ascii=False))
        data.append(json.dumps(body, ensure_ascii = False))
        #es_index.append(json.dumps({"index": {"_id": index_value, "_index": "cm"}}, ensure_ascii=False))
                
        if sys.getsizeof(data) > 100000:
            with open('/data/cm/output/es_temp_fra/chunk_'+str(chunk)+'.json', "w", encoding="utf-8") as write_file:
                #data = [json.dumps(d, ensure_ascii=False) for d in data]
                write_file.write("\n".join(data))
                
            chunk += 1
            data = []


Processing 2710055 Chunk 0 Processing 9616701 Chunk 0 Processing 2580773 Chunk 0 Processing 2895450 Chunk 0 Processing 2727810 Chunk 2 Processing 3129675 Chunk 2 Processing 6492429 Chunk 3 Processing 2438141 Chunk 3 Processing 2827798 Chunk 4 Processing 5438908 Chunk 4 Processing 4782723 Chunk 4 Processing 7506414 Chunk 5 Processing 9054153 Chunk 5 Processing 9366335 Chunk 5 Processing 4889892 Chunk 5 Processing 2425321 Chunk 5 Processing 6492329 Chunk 5 Processing 700673 Chunk 5 Processing 4895413 Chunk 5 Processing 2542917 Chunk 5 Processing 9583346 Chunk 6 Processing 2549643 Chunk 6 Processing 10638461 Chunk 6 Processing 3139318 Chunk 6 Processing 9620169 Chunk 6 Processing 9051331 Chunk 6 Processing 4781937 Chunk 6 Processing 4911569 Chunk 6 Processing 2625941 Chunk 6 Processing 2446951 Chunk 7 Processing 9643693 Chunk 20 Processing 2912821 Chunk 20 Processing 6465881 Chunk 21 Processing 9643701 Chunk 21 Processing 3020846 Chunk 21 Processing 9230930 Chunk 25 Processing 2644300 Chu

Processing 2420797 Chunk 134 Processing 4804947 Chunk 134 Processing 2609281 Chunk 134 Processing 3122061 Chunk 135 Processing 2424658 Chunk 136 Processing 4926451 Chunk 136 Processing 2571117 Chunk 136 Processing 2708691 Chunk 136 Processing 8812431 Chunk 136 Processing 9215017 Chunk 136 Processing 2420973 Chunk 136 Processing 2550047 Chunk 137 Processing 4926100 Chunk 137 Processing 3138070 Chunk 137 Processing 4862473 Chunk 137 Processing 8003959 Chunk 137 Processing 2821022 Chunk 137 Processing 2416673 Chunk 137 

In [ ]:
# instantiate elasticsearch
mappings = {
    "properties": {
        "text":   { "type": "text"  },
        "dateIssued": { "type": "text"},
        "lang": { "type": "text"},
        "place": { "type": "text"},
        "j_title": { "type": "text"},
        "publisher": { "type": "text"},
        "volume": { "type": "text"},
        "heft": { "type": "text"},
        "aufsatz": { "type": "text"},
        "page": { "type": "text"},
        "vlid_journal": { "type": "text"},
        "vlid_page": {"type": "text"}
        
    }
  }

In [19]:
es = elasticsearch.Elasticsearch()
ic = elasticsearch.client.IndicesClient(es)
if ic.exists('cm_meta'):
    ic.delete('cm_meta')
ic.create(index='cm_meta', body={'mappings': mappings})

{'acknowledged': True, 'shards_acknowledged': True, 'index': 'cm_meta'}

In [20]:
start = time.time()
for file in os.listdir('/data/cm/output/es_temp_fra/'):
    print(file)
    
    data = open('/data/cm/output/es_temp_fra/'+file, "r", encoding="utf-8").read()
    
    es_index = es.bulk(data, request_timeout=200)

end = time.time()
print("Indexing time:", str(end-start))

chunk_73.json
chunk_104.json
chunk_30.json
chunk_50.json
chunk_121.json
chunk_12.json
chunk_69.json
chunk_115.json
chunk_82.json
chunk_40.json
chunk_2.json
chunk_68.json
chunk_43.json
chunk_32.json
chunk_28.json
chunk_58.json
chunk_109.json
chunk_22.json
chunk_116.json
chunk_85.json
chunk_56.json
chunk_89.json
chunk_42.json
chunk_46.json
chunk_90.json
chunk_124.json
chunk_39.json
chunk_128.json
chunk_133.json
chunk_99.json
chunk_94.json
chunk_14.json
chunk_60.json
chunk_18.json
chunk_49.json
chunk_33.json
chunk_87.json
chunk_64.json
chunk_25.json
chunk_53.json
chunk_5.json
chunk_97.json
chunk_91.json
chunk_84.json
chunk_120.json
chunk_63.json
chunk_86.json
chunk_6.json
chunk_4.json
chunk_125.json
chunk_81.json
chunk_59.json
chunk_31.json
chunk_76.json
chunk_62.json
chunk_24.json
chunk_13.json
chunk_114.json
chunk_126.json
chunk_77.json
chunk_70.json
chunk_74.json
chunk_16.json
chunk_101.json
chunk_105.json
chunk_103.json
chunk_37.json
chunk_41.json
chunk_110.json
chunk_88.json
chunk_21

In [22]:
# query
es = elasticsearch.Elasticsearch()
start = time.time()
res = es.search(index='cm_meta', body={"query": {"match": {'text': 'Turin'}}})
end = time.time()
print("Search took", round(end-start, 2))

Search took 0.11


In [23]:
print(res.keys())
print(len(res['hits']['hits']))

dict_keys(['took', 'timed_out', '_shards', 'hits'])
10


In [24]:
print("Total hits:", res['hits']['total']['value'])
for doc in res['hits']['hits']:
    #print("%s) %s" % (doc['_id'], doc['_source']['text']))
    print("Id:", doc['_id'])
    print("Index:", doc['_index'])
    print("Score:", doc['_score'])
    print("Text:", doc['_source']['text'][:100])
    print("Date Issued:", doc['_source']['dateIssued'])
    print("Language:", doc['_source']['lang'])
    print("Place:", doc['_source']['place'])
    print("Journal Title:", doc['_source']['j_title'])
    print("Publisher:", doc['_source']['publisher'])
    print("Volume", doc['_source']['volume'])
    print("Heft", doc['_source']['heft'])
    print("Aufsatz", doc['_source']['aufsatz'])
    print("Page", doc['_source']['page'])
    print("VLID_journal", doc['_source']['vlid_journal'])
    print("VLID_page", doc['_source']['vlid_page'])
    
    break

Id: 07719_2947283
Index: cm_meta
Score: 11.805995
Text: 596 Die Rabbknen in Venedig und Turin wohnen unweit oder in den ehemaligen Ghettis, die Gottlob nunm
Date Issued: 1862-09-01
Language: ger
Place: Frankfurt, M.
Journal Title: Jeschurun
Publisher: Baist
Volume 8 (1835)
Heft Heft 12 (September 1862)
Aufsatz Italien : Appercu eines Reisenden ... [Nachricht]
Page 596
VLID_journal 2932754
VLID_page 2947283
